https://www.kaggle.com/datasets/anthonypino/melbourne-housing-market?select=Melbourne_housing_FULL.csv

Recommender system 
Discretized price - explicit/ implicit feedback in the scale of 1-5, 1-least expensive- 5-most expensive
User - SellerG
Item - Address

	Column_names	Description
0	Suburb	Suburb
1	Address	Address
2	Rooms	Number of rooms
3	Type	Type br - bedroom(s); h - house,cottage,villa, semi,terrace; u - unit, duplex; t - townhouse; dev site - development site; o res - other residential.
4	Price	Price in Australian dollars
5	Method	Method: S - property sold; SP - property sold prior; PI - property passed in; PN - sold prior not disclosed; SN - sold not disclosed; NB - no bid; VB - vendor bid; W - withdrawn prior to auction; SA - sold after auction; SS - sold after auction price not ...
6	SellerG	Real Estate Agent
7	Date	Date sold
8	Distance	Distance from CBD in Kilometres
9	Postcode	Postcode
10	Bedroom2	Scraped # of Bedrooms (from different source)
11	Bathroom	Number of Bathrooms
12	Car	Number of carspots
13	Landsize	Land Size in Metres
14	BuildingArea	Building Size in Metres
15	YearBuilt	Year the house was built
16	CouncilArea	Governing council for the area
17	Lattitude	Lattitude
18	Longtitude	Longtitude
19	Regionname	General Region (West, North West, North, North east …etc)
20	Propertycount	Number of properties that exist in the suburb.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import graphviz
from scipy.stats import boxcox
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import statsmodels.graphics.tsaplots as smgt
from statsmodels.stats import stattools
from sklearn.preprocessing import StandardScaler,Normalizer,LabelEncoder,PolynomialFeatures,MinMaxScaler
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV,StratifiedKFold,learning_curve,KFold,RandomizedSearchCV
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet,SGDRegressor
from sklearn.tree import DecisionTreeRegressor,export_graphviz
from sklearn.svm import LinearSVR
from sklearn.feature_selection import SelectFromModel, RFE,SelectPercentile,f_regression,VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy import stats
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
df= pd.read_csv('Melbourne_housing_FULL.csv')

In [3]:
df.head(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [5]:
df['Quantity'] = 1 

In [6]:
#Remove those redundant attributes 
#df.drop(['Address','Date','Postcode','YearBuilt','Lattitude','Longtitude'],axis=1,inplace=True)

In [7]:
#Continuous features 
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Rooms,34857.0,3.031012e+00,0.969933,1.00000,2.00000,3.0000,4.000000e+00,1.600000e+01
Price,27247.0,1.050173e+06,641467.130105,85000.00000,635000.00000,870000.0000,1.295000e+06,1.120000e+07
Distance,34856.0,1.118493e+01,6.788892,0.00000,6.40000,10.3000,1.400000e+01,4.810000e+01
Postcode,34856.0,3.116063e+03,109.023903,3000.00000,3051.00000,3103.0000,3.156000e+03,3.978000e+03
Bedroom2,26640.0,3.084647e+00,0.980690,0.00000,2.00000,3.0000,4.000000e+00,3.000000e+01
Bathroom,26631.0,1.624798e+00,0.724212,0.00000,1.00000,2.0000,2.000000e+00,1.200000e+01
Car,26129.0,1.728845e+00,1.010771,0.00000,1.00000,2.0000,2.000000e+00,2.600000e+01
Landsize,23047.0,5.935990e+02,3398.841946,0.00000,224.00000,521.0000,6.700000e+02,4.330140e+05
BuildingArea,13742.0,1.602564e+02,401.267060,0.00000,102.00000,136.0000,1.880000e+02,4.451500e+04
YearBuilt,15551.0,1.965290e+03,37.328178,1196.00000,1940.00000,1970.0000,2.000000e+03,2.106000e+03


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         34857 non-null  object 
 1   Address        34857 non-null  object 
 2   Rooms          34857 non-null  int64  
 3   Type           34857 non-null  object 
 4   Price          27247 non-null  float64
 5   Method         34857 non-null  object 
 6   SellerG        34857 non-null  object 
 7   Date           34857 non-null  object 
 8   Distance       34856 non-null  float64
 9   Postcode       34856 non-null  float64
 10  Bedroom2       26640 non-null  float64
 11  Bathroom       26631 non-null  float64
 12  Car            26129 non-null  float64
 13  Landsize       23047 non-null  float64
 14  BuildingArea   13742 non-null  float64
 15  YearBuilt      15551 non-null  float64
 16  CouncilArea    34854 non-null  object 
 17  Lattitude      26881 non-null  float64
 18  Longti

Missing values 

In [9]:
#simplificity, missing values > 40% of the dataset, remove them. Imputing them with mean, mode, median might impose a data skewness
df.isnull().sum()

Suburb               0
Address              0
Rooms                0
Type                 0
Price             7610
Method               0
SellerG              0
Date                 0
Distance             1
Postcode             1
Bedroom2          8217
Bathroom          8226
Car               8728
Landsize         11810
BuildingArea     21115
YearBuilt        19306
CouncilArea          3
Lattitude         7976
Longtitude        7976
Regionname           3
Propertycount        3
Quantity             0
dtype: int64

In [10]:
df.dropna(inplace= True)

In [11]:
df.isnull().sum()

Suburb           0
Address          0
Rooms            0
Type             0
Price            0
Method           0
SellerG          0
Date             0
Distance         0
Postcode         0
Bedroom2         0
Bathroom         0
Car              0
Landsize         0
BuildingArea     0
YearBuilt        0
CouncilArea      0
Lattitude        0
Longtitude       0
Regionname       0
Propertycount    0
Quantity         0
dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8887 entries, 2 to 34856
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         8887 non-null   object 
 1   Address        8887 non-null   object 
 2   Rooms          8887 non-null   int64  
 3   Type           8887 non-null   object 
 4   Price          8887 non-null   float64
 5   Method         8887 non-null   object 
 6   SellerG        8887 non-null   object 
 7   Date           8887 non-null   object 
 8   Distance       8887 non-null   float64
 9   Postcode       8887 non-null   float64
 10  Bedroom2       8887 non-null   float64
 11  Bathroom       8887 non-null   float64
 12  Car            8887 non-null   float64
 13  Landsize       8887 non-null   float64
 14  BuildingArea   8887 non-null   float64
 15  YearBuilt      8887 non-null   float64
 16  CouncilArea    8887 non-null   object 
 17  Lattitude      8887 non-null   float64
 18  Longtit

In [15]:
total_sales = df
total_sales['Price_visualization'] = total_sales['Price'] * total_sales['Quantity']
total_sales_sellerG_regionname = total_sales.groupby(['SellerG','Regionname']).agg({'Price_visualization':'sum'})
total_sales_sellerG_regionname.head()

Price_visualization
SellerG      Regionname                                
@Realty      Eastern Metropolitan             1650000.0
             Northern Metropolitan            1709000.0
Abercromby's Northern Metropolitan            2655000.0
             Southern Metropolitan           26124000.0
Ace          Western Metropolitan              600000.0

In [17]:
total_sales_sellerG_regionname.to_csv('market_basket_subset.csv')

In [20]:
total_sales_sellerG_regionname.reset_index(inplace=True)
total_sales_sellerG_regionname.sort_values(by = "Price_visualization", ascending = False).head(10)

,SellerG,Regionname,Price_visualization
231,Marshall,Southern Metropolitan,855409638.0
185,Jellis,Southern Metropolitan,674933457.0
269,Nelson,Northern Metropolitan,586830250.0
55,Buxton,Southern Metropolitan,441067750.0
443,hockingstuart,Southern Metropolitan,366536698.0
183,Jellis,Northern Metropolitan,344794341.0
272,Nelson,Western Metropolitan,343348200.0
20,Barry,Eastern Metropolitan,206758089.0
26,Barry,Western Metropolitan,194372200.0
22,Barry,Northern Metropolitan,172005776.0


Exploratory Data Analysis (EDA) 

In [30]:
# consider that for all time period
import plotly.express as px
data_fig = total_sales_sellerG_regionname.sort_values(by = "Price_visualization", ascending = False).head(15)

fig = px.scatter(data_fig, x="SellerG", y="Price_visualization",
        size="Price_visualization", color="Regionname",
        hover_name="Regionname", log_x=False, size_max=60, title="Top 10 SellerG and their Regionname")
fig.show()

Seller - Regionname Matrix 

In [51]:
basket = (df.groupby(['SellerG', 'Regionname'])['Quantity']
           .sum().unstack().reset_index().fillna(0)
           .set_index('SellerG'))
basket.head(10)

Regionname,Eastern Metropolitan,Eastern Victoria,Northern Metropolitan,Northern Victoria,South-Eastern Metropolitan,Southern Metropolitan,Western Metropolitan,Western Victoria
SellerG,,,,,,,,
@Realty,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
Abercromby's,0.0,0.0,2.0,0.0,0.0,10.0,0.0,0.0
Ace,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Alexkarbon,0.0,0.0,15.0,0.0,0.0,0.0,12.0,0.0
Allens,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Anderson,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
Appleby,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aquire,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
Area,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0


In [64]:
###Regionname as the basket 
basket_1 = (df.groupby(['Regionname','SellerG'])['Quantity']
           .sum().unstack().reset_index().fillna(0)
           .set_index('Regionname'))
basket_1.head(10)

SellerG,@Realty,Abercromby's,Ace,Alexkarbon,Allens,Anderson,Appleby,Aquire,Area,Ascend,...,buyMyplace,hockingstuart,hockingstuart/Advantage,hockingstuart/Biggin,hockingstuart/Village,hockingstuart/hockingstuart,iOne,iProperty,iSell,iTRAK
Regionname,,,,,,,,,,,,,,,,,,,,,
Eastern Metropolitan,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,...,1.0,41.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0
Eastern Victoria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
Northern Metropolitan,3.0,2.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,149.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Northern Victoria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
South-Eastern Metropolitan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,...,0.0,30.0,0.0,0.0,0.0,3.0,0.0,0.0,7.0,1.0
Southern Metropolitan,0.0,10.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,...,1.0,328.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Western Metropolitan,0.0,0.0,1.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,113.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
Western Victoria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
def encode_units(x):
     if x <= 0:
             return 0
     if x >= 1:
             return 1
basket_sets = basket.applymap(encode_units)


In [65]:
def encode_units(x):
     if x <= 0:
             return 0
     if x >= 1:
             return 1
basket_sets_1 = basket_1.applymap(encode_units)

In [54]:
basket_sets.head()

Regionname,Eastern Metropolitan,Eastern Victoria,Northern Metropolitan,Northern Victoria,South-Eastern Metropolitan,Southern Metropolitan,Western Metropolitan,Western Victoria
SellerG,,,,,,,,
@Realty,1,0,1,0,0,0,0,0
Abercromby's,0,0,1,0,0,1,0,0
Ace,0,0,0,0,0,0,1,0
Alexkarbon,0,0,1,0,0,0,1,0
Allens,1,0,0,0,0,0,0,0


In [66]:
basket_sets_1.head()

SellerG,@Realty,Abercromby's,Ace,Alexkarbon,Allens,Anderson,Appleby,Aquire,Area,Ascend,...,buyMyplace,hockingstuart,hockingstuart/Advantage,hockingstuart/Biggin,hockingstuart/Village,hockingstuart/hockingstuart,iOne,iProperty,iSell,iTRAK
Regionname,,,,,,,,,,,,,,,,,,,,,
Eastern Metropolitan,1,0,0,0,1,0,1,0,0,0,...,1,1,0,0,0,0,0,1,0,1
Eastern Victoria,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
Northern Metropolitan,1,1,0,1,0,0,0,0,0,1,...,1,1,1,0,0,0,0,0,0,1
Northern Victoria,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
South-Eastern Metropolitan,0,0,0,0,0,0,0,1,1,0,...,0,1,0,0,0,1,0,0,1,1


Association Rules Learning

In [35]:
!pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

  Using cached mlxtend-0.22.0-py2.py3-none-any.whl (1.4 MB)


C:\Users\looi\AppData\Local\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



,support,itemsets
5,0.412,"((Price, Southern Metropolitan))"
6,0.392,"((Price, Western Metropolitan))"
2,0.352,"((Price, Northern Metropolitan))"
0,0.248,"((Price, Eastern Metropolitan))"
4,0.228,"((Price, South-Eastern Metropolitan))"
...,...,...
106,0.004,"((Price, Eastern Victoria), (Price, Eastern Me..."
180,0.004,"((Price, Western Metropolitan), (Price, Wester..."
220,0.004,"((Price, Western Victoria), (Price, Eastern Vi..."
222,0.004,"((Price, Western Victoria), (Price, Eastern Vi..."


In [55]:
frequent_itemsets = apriori(basket_sets, min_support=0.001, use_colnames=True) 
frequent_itemsets.sort_values("support", ascending=False)

C:\Users\looi\AppData\Local\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



,support,itemsets
5,0.412,(Southern Metropolitan)
6,0.392,(Western Metropolitan)
2,0.352,(Northern Metropolitan)
0,0.248,(Eastern Metropolitan)
4,0.228,(South-Eastern Metropolitan)
...,...,...
106,0.004,"(Western Victoria, Eastern Metropolitan, Easte..."
180,0.004,"(Eastern Metropolitan, Western Victoria, Easte..."
220,0.004,"(Eastern Metropolitan, Northern Victoria, West..."
222,0.004,"(Eastern Metropolitan, Northern Victoria, West..."


In [ ]:
frequent_itemsets = apriori(basket_sets_1, min_support=0.001, use_colnames=True) 
frequent_itemsets.sort_values("support", ascending=False)

In [56]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.001)
rules.sort_values("support", ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
31,(Southern Metropolitan),(Northern Metropolitan),0.412,0.352,0.176,0.427184,1.213592,0.030976,1.131254,0.299320
30,(Northern Metropolitan),(Southern Metropolitan),0.352,0.412,0.176,0.500000,1.213592,0.030976,1.176000,0.271605
32,(Northern Metropolitan),(Western Metropolitan),0.352,0.392,0.160,0.454545,1.159555,0.022016,1.114667,0.212346
33,(Western Metropolitan),(Northern Metropolitan),0.392,0.352,0.160,0.408163,1.159555,0.022016,1.094897,0.226316
50,(Southern Metropolitan),(Western Metropolitan),0.412,0.392,0.132,0.320388,0.817317,-0.029504,0.894629,-0.275429
51,(Western Metropolitan),(Southern Metropolitan),0.392,0.412,0.132,0.336735,0.817317,-0.029504,0.886523,-0.268805
44,(South-Eastern Metropolitan),(Southern Metropolitan),0.228,0.412,0.124,0.543860,1.320048,0.030064,1.289077,0.314056
45,(Southern Metropolitan),(South-Eastern Metropolitan),0.412,0.228,0.124,0.300971,1.320048,0.030064,1.104389,0.412333
2,(Northern Metropolitan),(Eastern Metropolitan),0.352,0.248,0.116,0.329545,1.328812,0.028704,1.121627,0.381865
3,(Eastern Metropolitan),(Northern Metropolitan),0.248,0.352,0.116,0.467742,1.328812,0.028704,1.217455,0.329054


In [57]:
sorted_rules = rules.sort_values("lift", ascending=False)
rules.sort_values("lift", ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4086,"(Western Victoria, Eastern Metropolitan)","(South-Eastern Metropolitan, Northern Metropol...",0.008,0.028,0.008,1.0,35.714286,0.007776,inf,0.979839
2041,"(Western Victoria, Eastern Metropolitan, North...","(South-Eastern Metropolitan, Northern Victoria)",0.008,0.028,0.008,1.0,35.714286,0.007776,inf,0.979839
3189,"(Western Victoria, Eastern Metropolitan, North...","(South-Eastern Metropolitan, Northern Victoria)",0.004,0.028,0.004,1.0,35.714286,0.003888,inf,0.975904
5507,"(Western Victoria, Eastern Victoria, South-Eas...","(Eastern Metropolitan, Southern Metropolitan, ...",0.004,0.028,0.004,1.0,35.714286,0.003888,inf,0.975904
4148,"(Western Victoria, Eastern Metropolitan)","(South-Eastern Metropolitan, Northern Metropol...",0.008,0.028,0.008,1.0,35.714286,0.007776,inf,0.979839
2048,"(Western Victoria, Eastern Metropolitan)","(South-Eastern Metropolitan, Northern Metropol...",0.008,0.028,0.008,1.0,35.714286,0.007776,inf,0.979839
5108,"(Western Victoria, Eastern Metropolitan, Easte...","(South-Eastern Metropolitan, Northern Metropol...",0.004,0.028,0.004,1.0,35.714286,0.003888,inf,0.975904
2311,"(Western Victoria, Eastern Metropolitan, South...","(South-Eastern Metropolitan, Northern Victoria)",0.008,0.028,0.008,1.0,35.714286,0.007776,inf,0.979839
5995,"(Western Victoria, Eastern Victoria, South-Eas...","(Eastern Metropolitan, Northern Victoria, West...",0.004,0.028,0.004,1.0,35.714286,0.003888,inf,0.975904
5456,"(Western Victoria, Eastern Metropolitan, Easte...","(South-Eastern Metropolitan, Southern Metropol...",0.004,0.028,0.004,1.0,35.714286,0.003888,inf,0.975904


In [49]:
# Assuming you have a DataFrame called 'df' with columns 'SellerG', 'Regionname', and 'Price'

# Group by 'SellerG' and 'Regionname', calculate sum of 'Price' for each group
grouped_df = df.groupby(['SellerG', 'Regionname']).agg({"Price": "sum"})

# Create a matrix representation
df_seller_regionname_matrix_1 = grouped_df.unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)

# Print the resulting matrix
print(df_seller_regionname_matrix_1.head(10))

df_seller_regionname_matrix_1.columns = df_seller_regionname_matrix.columns.get_level_values(1)

print(df_seller_regionname_matrix_1.head(10))

                            Price                                         \
Regionname   Eastern Metropolitan Eastern Victoria Northern Metropolitan   
SellerG                                                                    
@Realty                         1                0                     1   
Abercromby's                    0                0                     1   
Ace                             0                0                     0   
Alexkarbon                      0                0                     1   
Allens                          1                0                     0   
Anderson                        0                0                     0   
Appleby                         1                0                     0   
Aquire                          0                0                     0   
Area                            0                0                     0   
Ascend                          0                0                     1   

           

In [32]:
df_seller_regionname_matrix = df.groupby(['SellerG','Regionname']).  \
    agg({"Price": "sum"}).unstack().fillna(0).  \
    applymap(lambda x: 1 if x > 0 else 0)

df_seller_regionname_matrix.head(100)

Price                                         \
Regionname   Eastern Metropolitan Eastern Victoria Northern Metropolitan   
SellerG                                                                    
@Realty                         1                0                     1   
Abercromby's                    0                0                     1   
Ace                             0                0                     0   
Alexkarbon                      0                0                     1   
Allens                          1                0                     0   
...                           ...              ...                   ...   
J                               0                0                     1   
JMRE                            0                0                     1   
JRW                             1                0                     0   
Jas                             0                0                     0   
Jason                           0                0                     1   

                                                           \
Regionname   Northern Victoria South-Eastern Metropolitan   
SellerG                                                     
@Realty                      0                          0   
Abercromby's                 0                          0   
Ace                          0                          0   
Alexkarbon                   0                          0   
Allens                       0                          0   
...                        ...                        ...   
J                            0                          0   
JMRE                         0                          0   
JRW                          0                          0   
Jas                          0                          0   
Jason                        0                          0   

                                                                          
Regionname   Southern Metropolitan Western Metropolitan Western Victoria  
SellerG                                                                   
@Realty                          0                    0                0  
Abercromby's                     1                    0                0  
Ace                              0                    1                0  
Alexkarbon                       0                    1                0  
Allens                           0                    0                0  
...                            ...                  ...              ...  
J                                1                    0                0  
JMRE                             0                    0                0  
JRW                              0                    0                0  
Jas                              0                    1                0  
Jason                            0                    1                0  

[100 rows x 8 columns]